# Using circuits designed with external NISQ frameworks

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from pathlib import Path

ASSETS_FOLDER = Path("../../assets/").resolve()


# To display interactive 3D visualisations, uncomment the following line.
# %matplotlib widget

This notebook illustrates the process of importing into TQEC circuits originally created with external NISQ circuit design software/frameworks, such as, for instance, Qiskit and Qrisp.

> **NB!** The notebook uses Qiskit by default, which installs with TQEC. To use a different framework, run `uv add --only-group integration` to install all supported framework libraries or `uv add <chosen_framework>` to install only the desired framework.

## Load the foundational circuit

The first step is, naturally, to load your foundational circuit. This can be achieved easily by calling a TQEC pipeline that reads and convert a circuit in the native format of any supported framework and converts it into an OpenQASM version of itself.

In [ ]:
from tqec.interop.nisq.load import load_nisq_circuit_as_qasm

source_nisq_framework = "qiskit"  # options currently available ["qiskit", "qrisp"]
path_to_source_c = ASSETS_FOLDER / "steane.qpy"

qasm_str = load_nisq_circuit_as_qasm(path_to_source_c, source_nisq_framework, display_source_c=True)

And let's not forget to inspect the QASM string before moving on.

In [ ]:
print(qasm_str)

## QASM to PyZX

To be able to perform algorithmic lattice surgery on a QASM string, we need to import it into PyZX and convert it into a PyZX graph. 

So, let's start by turning `qasm_str` into a PyZX circuit.

In [ ]:
import pyzx as zx
zx.settings.colors = zx.rgb_colors

# Import QASM into PyZX
zx_circuit = zx.Circuit.from_qasm(qasm_str)

# Convert incoming circuit into graph
zx_graph = zx_circuit.to_graph()

# Draw graph of incoming circuit
zx.draw(zx_graph, labels = True)

The incoming circuit is not optimised. Topologiq can handle an unoptimised circuit, but the space-time volume of the diagram produced by topologiq is increasingly proportional to the size of the input graph. It is therefore a good idea to optimise the incoming circuit. 

For this, since PyZX's QASM compatibility is still in-development, we need to start by initialising the qubits after converting the circuit into a graph and adding measurement bases for the ancilla qubits.

In [ ]:
# Apply states
num_apply_state = zx_graph.num_inputs()
zx_graph.apply_state('0' * num_apply_state)

# Apply post-select only to the outputs of the ancilla qubits
zx_graph.apply_effect('000///////')
zx.draw(zx_graph, labels = True)

It is now possible to use more-standard PyZX methods to optimise the circuit/graph further.

In [ ]:
# From Aleks Kissinger's notebook: https://nbviewer.org/github/zxcalc/pyzx/blob/master/demos/example-circuit-simp.ipynb
zx.full_reduce(zx_graph)
zx.to_rg(zx_graph)

random.seed(12)
zx.draw(zx_graph, labels = True, auto_layout=True)

# Object below is not typical in a PyZX workflow but useful for visualisations in the next section.
fig_data = zx.draw_matplotlib(zx_graph, labels=True)

## Algorithmic lattice surgery

It is now possible to convert this graph into `block_graph` compatible objects by giving it to topologiq. 

The first is to convert the graph into topologiq's native graph object, a `simple_graph`.

In [ ]:
from topologiq.scripts.runner import runner
from topologiq.utils.interop_pyzx import pyzx_g_to_simple_g

# Transform ZX-graph into topologiq's native `simple_graph`
simple_graph = pyzx_g_to_simple_g(zx_graph)

# Print `simple_graph for inspection`
for k, v in simple_graph.items():
        print(f"{k}: {v}")

After, we can run the `simple_graph` by topologiq.

In [ ]:
# Parameters & hyper-parameters
circuit_name = f"{source_nisq_framework}_steane"
visualisation = "final"  # Calls 3D visualisation at the end. `None` to deactivate.
animation = None  # Change to "GIF" or "MP4" for a summary animation (significant runtime costs).

VALUE_FUNCTION_HYPERPARAMS = (
    -1,  # Weight for length of path
    -1,  # Weight for number of "beams" broken by path
)

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": 9,
}

# Run topologiq
simple_graph_after_use, edge_pths, lattice_nodes, lattice_edges = runner(
    simple_graph,  # The PyZX input graph, simplified
    circuit_name,  # The name of the circuit
    visualise=(visualisation, animation),
    fig_data=fig_data,
    **kwargs
)

And we can print topologiq's outputs to inspect them, in particular, `lattice_nodes` and `lattice_edges`, which will become a TQEC `block_graph`.

In [ ]:
if lattice_nodes and lattice_edges:
    print("\nCubes in final output:")
    for k, v in lattice_nodes.items():
        print(f"{k}:{v}")

    print("\nPipes in final output:")
    for k, v in lattice_edges.items():
        print(f"{k}:{v}")

else:
    print("WARNING! Some key objects needed to create the block_graph do not exist. Please check that topologiq ran and succeeded.")

## Importing to TQEC

The structure of `lattice_nodes` and `lattice_edges` is essentially the same as TQEC's `block_graph`, with one caveat.

There is a need to perform some re-scaling and re-indexing of edges. In TQEC's `block_graph`, pipes occupy no space, which leads to adjacent cubes being closer to one another than in a space-time diagram where pipes do occupy space for the purposes of visualising things clearly.

Fortunately, TQEC already has a way to import 3D objects like `lattice_nodes` and `lattice_edges`. TQEC can import 3D models encoded in COLLADA format (.dae), which use the same kind of coordinates and artifacts as `lattice_nodes` and `lattice_edges`. Accordingly, the following block of code relies on TQEC's COLLADA interoperability to realise similar transformations for `lattice_nodes` and `lattice_edges`, leading into a `block_graph`.

In [ ]:
from tqec.interop.pyzx.topologiq import read_from_lattice_dicts

if lattice_nodes and lattice_edges:

    # Import using standardised method for importing lattice surgeries / space-time diagram into TQEC
    lattice_edges_min = dict([(k,v[0]) for k,v in lattice_edges.items()])
    block_graph = read_from_lattice_dicts(lattice_nodes, lattice_edges_min, graph_name=circuit_name)
    
    # Visualise using TQEC methods
    html = block_graph.view_as_html()
    display(html)

You may have noticed the imported blockgraph lacks the boundary nodes / port cubes, the ones seen in gray in topologiq's 3D visualisation. 

This is normal. The TQEC import adds transparent placeholder objects at those coordinates.

To enable computation, these need to be "filled", which is done as follows.

In [ ]:
if block_graph:
    
    # Fill ports using function that yields minimal set of block_graphs needed for simulation
    filled_block_graphs = block_graph.fill_ports_for_minimal_simulation()
    
    # Show all block_graphs in minimal set
    for i, block_graph in enumerate(filled_block_graphs):

        for j, correlation_surface in enumerate(block_graph.observables):
            block_graph.graph.view_as_html(
                pop_faces_at_directions=("-Y", "+X"),
                show_correlation_surface=block_graph.observables[j],
            )

            html = block_graph.graph.view_as_html(
                pop_faces_at_directions=("-Y", "+X"),
                show_correlation_surface=block_graph.observables[j],
            )

            display(html)

And we can now use the blockgraph to perform TQEC operations. 

Let's start by getting the Stim circuit for different bases. 

In [ ]:
import sinter
from tqec.computation.block_graph import BlockGraph
from tqec import compile_block_graph, NoiseModel
from tqec.utils.enums import Basis
from tqec.simulation.plotting.inset import plot_observable_as_inset
from tqec.simulation.simulation import start_simulation_using_sinter

# Function to get the correct filled_graphs for each basis
def graphs_for_given_basis(pre_filled_block_graphs, observable_basis: Basis) -> BlockGraph | None:
    
    filled_graphs = pre_filled_block_graphs
    assert len(filled_graphs) == 2
    if observable_basis == Basis.X:
        return filled_graphs[0].graph
    elif observable_basis == Basis.Z:
        return filled_graphs[1].graph

# Function to get Stim circuit for given basis
def get_stim_circuit(pre_filled_block_graphs):

    block_graph_for_computation = graphs_for_given_basis(pre_filled_block_graphs, Basis.X)
    if block_graph_for_computation:
        compiled_graph = compile_block_graph(block_graph_for_computation)
        stim_circuit = compiled_graph.generate_stim_circuit(
            k=1, noise_model=NoiseModel.uniform_depolarizing(p=0.001)
        )
        print("\nFirst 10 lines of Stim circuit (Basis X):\n")
        print(stim_circuit[:10])
    
    block_graph_for_computation = graphs_for_given_basis(pre_filled_block_graphs, Basis.Z)
    if block_graph_for_computation:
        compiled_graph = compile_block_graph(block_graph_for_computation)
        stim_circuit = compiled_graph.generate_stim_circuit(
            k=1, noise_model=NoiseModel.uniform_depolarizing(p=0.001)
        )
        print("\nFirst 10 lines of Stim circuit (Basis Z):\n")
        print(stim_circuit[:10])

# Call Stim circuit generation for all relevant bases
if filled_block_graphs:
    get_stim_circuit(filled_block_graphs)

And let's now run a simulation. 

In [ ]:
# Function to generate simulation graphs
def generate_graphs(pre_filled_block_graphs, support_observable_basis: Basis, SAVE_DIR) -> None:

    # Get block_graph from set of pre_filled block_graphs
    block_graph_for_simulation = graphs_for_given_basis(pre_filled_block_graphs, support_observable_basis)
    
    # Start sinter
    if block_graph_for_simulation:

        # ZX-graph to overlay over output
        zx_graph_for_simulation = block_graph_for_simulation.to_zx_graph()

        # Correlation surfaces for simulation
        correlation_surfaces_for_simulation = block_graph_for_simulation.find_correlation_surfaces()
        
        # Call Sinter
        stats = start_simulation_using_sinter(
            block_graph_for_simulation,
            range(1, 4),
            list(np.logspace(-4, -1, 10)),
            NoiseModel.uniform_depolarizing,
            manhattan_radius=2,
            observables=correlation_surfaces_for_simulation,
            max_shots=1_000_000,
            max_errors=5_000,
            decoders=["pymatching"],
            print_progress=False,
            save_resume_filepath=Path(
                f"../_examples_database/{circuit_name}_{support_observable_basis.value}.csv"
            ),
            database_path=Path("../_examples_database/database.pkl"),
        )

        # Visualise output
        for i, stat in enumerate(stats):
            fig, ax = plt.subplots()
            sinter.plot_error_rate(
                ax=ax,
                stats=stat,
                x_func=lambda stat: stat.json_metadata["p"],
                group_func=lambda stat: stat.json_metadata["d"],
            )
            plot_observable_as_inset(ax, zx_graph_for_simulation, correlation_surfaces_for_simulation[i])
            ax.grid(axis="both")
            ax.legend()
            ax.loglog()
            ax.set_title("Logical CNOT Error Rate")
            ax.set_xlabel("Physical Error Rate")
            ax.set_ylabel("Logical Error Rate")
            fig.savefig(
                SAVE_DIR /
                f"{circuit_name}_{support_observable_basis}_{i}.png"
            )

# Function to call the simulation cycle for all relevant bases
def run_simulation(pre_filled_block_graphs, SAVE_DIR):
    SAVE_DIR.mkdir(exist_ok=True)
    generate_graphs(pre_filled_block_graphs, Basis.Z, SAVE_DIR)
    generate_graphs(pre_filled_block_graphs, Basis.X, SAVE_DIR)

# Call simulation cycle
if filled_block_graphs:
    SAVE_DIR = Path("results")
    run_simulation(filled_block_graphs, SAVE_DIR)